# Monty Hall Problem

This notebook provides an interactive simulation of the Monty Hall Problem from probability
and game theory. The Monty Hall Problem is a game of chance with conditional information:

1. The game begins with a prize randomly hidden behind one of three doors.
2. The reader chooses an initial door out of the three.
3. A door that neither the reader has chosen, nor containing a prize is then opened.
4. The reader can then optionally choose to switch their choice of doors to the remaining
unopened door.
5. Finally the prize door is revealed. If it is the final choice of the reader then the
reader wins, otherwise the reader loses.

The counterintuitive result of this game is that the optimum strategy is for the reader to
always switch their choice of doors, as that has a two thirds chance of winning. In this
notebook the reader acts as the contest and can play multiple rounds of the game. When the
reader stops playing the wins and loses are cross-tabulated by the strategy the reader
chose.

In [1]:
import random as rd

## Interactive Simulation

We will take this opportunity to demonstrate Functional Programming in Python. The
interactive simulation will demonstrate the following Python concepts:

* Recursive functions
* Local functions
* `nonlocal` keyword
* Sets and their operations
* Incrementally updating variables
* Random choices
* Flattened conditional branching
* Returning values from a function

### Recursive Functions

Before we begin with the simulation, what will happen if a function calls itself? This is
called recursive logic or reasoning. This function will count down.

In [2]:
def countdown(n):
    """
    Counts down from the the argument `n` passed in, printing a message.
    """
    if n < 1:
        print("All done.")
    else:
        print(f"{n} to go.")
        countdown(n - 1)

In [3]:
countdown(6)

6 to go.
5 to go.
4 to go.
3 to go.
2 to go.
1 to go.
All done.


### Factory Functions

How do we make a recursive function count up. We have a couple of choices. First we could
use two arguments, one for the current count and one for the final goal. Or we could
use a protected global in a factory function.

Factory functions are just functions that return other functions. They define a new global
scope for the returned function, so that the same call to a function can produce different
results.

In [4]:
def factory():
    """
    Will return a function that recursively counts up. It is the job of the factory function
    to keep track of the engines progress in reaching a goal
    """
    progress = 0
    
    def engine(goal):
        """
        Starting from 1 count up to the passed in argument `goal`
        """

        # This gives us access to the variable progress defined in the outer scope.
        nonlocal progress
        progress +=1

        # Test if we have reach the goal, otherwise keep going
        if progress < goal:
            print(f"{progress} so far.")
            engine(goal)
        else:
            print("All done.")

    # Send out the engine that was built, to be used. No arguments provided.
    return engine

In [5]:
# First we create an instance of the count down
countup = factory()
print(type(countup))

# Then we use the function
countup(6)

<class 'function'>
1 so far.
2 so far.
3 so far.
4 so far.
5 so far.
All done.


Notice how calling `countup` again remembers the progress so far, even across the calls.

Calling with a larger value than the last goal will count upwards.

In [6]:
countup(12)

7 so far.
8 so far.
9 so far.
10 so far.
11 so far.
All done.


Calling with a smaller value than the progress will not count upwards because the progress
has already exceed the goal.

In [7]:
countup(3)

All done.


We can create a new version of `countup` to reset the progress.

In [8]:
countup = factory()
countup(7)

1 so far.
2 so far.
3 so far.
4 so far.
5 so far.
6 so far.
All done.


### Sets

A Python set ensures the elements in a set are unique. Sets are created with curly braces 
`{}`. Sets can be unpacked with the `*` operator, and turned into lists with `[]`. If you
create a set it will automatically remove duplicates. 

In [9]:
# Similar notation to dictionaries but with single elements instead of key-value pairs.
a = { 1, 2, 3}
print(a)

# Unpack to elements
print(*a)
print(1, 2, 3)

# Unpack to list
print([ *a ])

# De-duplication
print({ 2, 2 })

{1, 2, 3}
1 2 3
1 2 3
[1, 2, 3]
{2}


Using these techniques we can build a recursive version of the Monty Hall simulator.

In [19]:
def game():
    """
    The classic Monty Hall Game using text inputs. Returns a cross-tabulation of wins and
    loses by choice of swap versus stay:
    * `swapwins` - Number of wins after switching doors.
    * `swaploses` - Number of loses after switching doors.
    * `staywins` - Number of wins after keeping the choice.
    * `staylose` - Number of loses after keeping the choice.
    """

    # The doors as a Python set
    DOORS = { '1', '2', '3' }

    # Cross-tabulations of results
    swapwins = 0
    swaploses = 0
    staywins = 0
    stayloses = 0

    def engine():
        """
        Locally defined recursive game engine that runs until exit is chosen.
        """

        # Get access to the cross-tabulations to track between individual games
        nonlocal DOORS
        nonlocal swapwins
        nonlocal swaploses
        nonlocal staywins
        nonlocal stayloses

        # Put a prize randomly behind a door
        prize = rd.choice([ *DOORS ])

        # Prompt the user to choose a door
        first = input("Type '1', '2', or '3' to choose a door.")
        if first not in DOORS:
            return False
        
        # Select a door to reveal. Randomly select from the set of doors that neither have
        # the prize nor selected by the user. We can do this by making a set of the prize
        # and the users first choice and removing those from all the doors
        reveal = rd.choice([ *DOORS.difference({ prize, first }) ])

        # Prompt the user to stay or swap with the additional information.
        stay = input(
            f"The prize is not behind door {reveal}." +
            f"Your first choice was door {first}." +
            "Type 'y' to stay with this choice."
        )

        # Update the cross tabulation based on the choices
        if stay == "y" and prize == first:
            staywins += 1
            print("Win!")
        elif stay == "y":
            print("Lost.")
            stayloses += 1
        elif prize == first:
            swaploses += 1
            print("Lost.")
        else:
            swapwins += 1
            print("Win!")

        # Prompt to continue
        again = input("Type 'y' to play again.")
        return again == "y"

    # Initialize the game
    while engine():
        pass

    # Send the results
    return swapwins, swaploses, staywins, stayloses

In [20]:
game()

Win!
Win!
Win!
Lost.
Lost.
Win!
Lost.
Lost.
Lost.
Win!
Win!
Lost.
Win!


(7, 6, 0, 0)

## Factorial Recursive Function

The textbook demonstration of a recursive function is the factorial:
$$
n! = n \times n - 1 \times \cdots \times 1
$$
We can redefine this as:
$$
n! = n \times (n - 1)!
$$
With the condition that 0! = 1


In [7]:
def factorial(n):
    if n < 2:
        return 1
    return n * factorial(n-1)

In [ ]:
factorial(10)